# RDDs con PySpark

En este notebook aprenderemos cómo crear una sesión de Spark (SparkSession) y cómo crear RDDs a partir de colecciones de Python y de ficheros externos.

Para poder ejecutar Spark es necesario disponer de una instalación de Java. Se puede comprobar si Java está instalado en la máquina con el siguiente comando.

In [ ]:
!java --version

En caso de que el comando anterior falle, será necesario instalar Java en la máquina. Podemos instalar la versión LTS de Java 21 con el siguiente comando si estamos en Google Colab o en una máquina con Linux. Para instalar Java en Windows, tendremos que visitar la siguiente página (https://www.oracle.com/es/java/technologies/downloads)

In [ ]:
!apt install -y openjdk-21-jdk

A continuación, será necesario comprobar si la variable `JAVA_HOME`, que indica a Spark dónde encontrar la JVM para ejecutar su código Scala, está definida.

In [ ]:
import os

os.environ["JAVA_HOME"]

En caso de que la variable no esté definida, se debe establecer de la siguiente manera.

In [4]:
# En nuestro ordenador personal, si no esta definida la variable JAVA_HOME, deberemos indicarla
# Para sistemas basados en Debian/Ubuntu, si tenemos instalada la version 21 de Java, seria:
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-21-openjdk-amd64"
# En Windows, la ruta puede ser algo como: "C:\\Program Files\\Java\\jdk-21"
# os.environ["JAVA_HOME"] = "C:\\Program Files\\Java\\jdk-21"
# Si ya esta definida, no es necesario hacer nada
# os.environ["JAVA_HOME"]

# En Google Colab no es necesario hacer nada

A continuación, importaremos PySpark y verificaremos si está correctamente instalado.

In [ ]:
import pyspark

pyspark.__version__

'4.0.1'

En caso de que PySpark no esté instalado, se debe instalar con el siguiente comando.

In [ ]:
!pip install pyspark

import pyspark
pyspark.__version__

Ya estamos listos para crear una sesión de Spark y empezar a trabajar con RDDs.

In [18]:
from pyspark.sql import SparkSession

# Crear sesión de Spark
spark = SparkSession.builder.appName("Analisis Deportivo con Spark").getOrCreate()

Una vez creado un objeto `SparkSession`, podemos inspeccionarlo. Ahora mismo lo estamos utilizando en modo local, con todos los núcleos de nuestro ordenador -> [*].

In [3]:
spark

### Ejemplo 1

Vamos a crear un RDD a partir de una colección de Python. En este caso, una lista con información sobre jugadores y su distancia recorrida en un partido de fútbol.

In [20]:
# Datos simulados: (jugador, distancia recorrida en km)
football_data = [("Jugador1", 10.2), ("Jugador2", 9.8), ("Jugador1", 11.0)]

# Creamos un RDD a partir de la lista
rdd = spark.sparkContext.parallelize(football_data)

# Distancia total por jugador
total_distance = rdd.reduceByKey(lambda a, b: a + b)
total_distance.collect()

[('Jugador2', 9.8), ('Jugador1', 21.2)]

### Ejemplo 2

Veremos una implementación del clásico problema de contar palabras (word count) utilizando RDDs. En este caso, el RDD se crea a partir de un fichero de texto.

Si estamos trabajando en local con Jupyter Notebook/Lab, accedemos directamente utilizando la carpeta `data/` (deberás comentar las opciones de Google Collab para que no sobrescriba)

In [23]:
file_path = "data/wordcount_data.txt"

Si estamos en Google Collab tenemos 2 opciones:

**Opción 1: Subir un fichero de datos a mano:**
- Abrirmos el directorio actual (:file_folder:) y arrastramos el fichero que queremos utilizar (wordcount_data.txt)
- Cuando cerremos Google Colab, ese fichero se _perderá_
- El fichero estará disponible usando simplemente su nombre

In [ ]:
file_path = "wordcount_data.txt"

**Opción 2: Montamos una carpeta de Google Drive que contenga nuestros datos**
- Deberemos tener creada una carpeta "Colab Notebooks" en la raiz de nuestra carpeta de Google Drive (se crea sola al subir el primer Notebook)
- Subimos nuestros datos (por ejemplo, la carpeta `data/` que tenemos en los ejemplos de Spark)
- Para tener las rutas más limpias, creamos un enlace simbólico para que los datos estén disponibles en la carpeta workspace


In [ ]:
# Montamos la carpeta (nos pedirá permisos)
from google.colab import drive
drive.mount('/content/drive')
# Crea un atajo llamado 'workspace' en la carpeta /content (dará un pequeño error si ya existe)
!ln -s "/content/drive/MyDrive/Colab Notebooks" "/content/workspace" >/dev/null 2>&1
# Ya podemos acceder a los ficheros, por ejemplo:
file_path = "/content/workspace/data/wordcount_data.txt"

In [24]:
rdd = spark.sparkContext.textFile(file_path)

Mostramos el número de líneas del fichero

In [25]:
rdd.count() # Numero de lineas

44

In [26]:
rdd_word_count =(rdd
    .flatMap(lambda line: line.split())
    .map(lambda word: (word, 1))
    .reduceByKey(lambda x, y: x + y))

In [14]:
rdd_word_count.saveAsTextFile("output_wordcount") # Si ya existe el directorio, da error, podemos ejecutar en la terminal rm -rf output_wordcount/

In [27]:
rdd_word_count.takeOrdered(10, key=lambda x: -x[1])

[('the', 38),
 ('a', 28),
 ('of', 25),
 ('word', 24),
 ('and', 23),
 ('words', 21),
 ('is', 19),
 ('to', 18),
 ('count', 11),
 ('in', 11)]

Finalmente, podemos cerrar la sesión de Spark cuando ya no la necesitemos. Si lo hacemos, habrá que ejecutar todo el notebook de nuevo para volver a crearla.

In [28]:
spark.stop()